# Create csv file from pre processed text files:

In [6]:
import os
import pandas as pd

data_dir = 'dataset/price/preprocessed/'

for stock_name in os.listdir(data_dir):
    txt_stockprice_file = os.path.join(data_dir, stock_name)
    if txt_stockprice_file.endswith('txt'):
        csv_stockprice_file = os.path.join(data_dir, 'csv/', stock_name.replace('.txt', '.csv'))
        #csv_stockprice_file = f"csv/{txt_stockprice_file.replace('.txt', '.csv')}"
        df = pd.read_csv(
                    txt_stockprice_file, 
                    delimiter="\t", 
                    header=None, 
                    names=["Date", "Movement_Percent", "Open", "High", "Low", "Close", "Volume"]
                )
        
        df["Date"] = pd.to_datetime(df["Date"])
        df = df.sort_values(by="Date")

        df.to_csv(csv_stockprice_file, index=False)



### Convert tweets files to proper json

In [13]:
import os
import json

# Input folder containing the improperly formatted JSON files
input_folder = "dataset/tweet/preprocessed"
output_root = "dataset/tweet/preprocessed-json/"



for stock in os.listdir(input_folder):
    stock_folder = os.path.join(input_folder, stock)
    for date in os.listdir(stock_folder):
        output_folder = os.path.join(output_root, stock)
        input_file = os.path.join(input_folder, stock, date)
        if date.startswith("20"):
            os.makedirs(output_folder, exist_ok=True)
            output_file = os.path.join(output_folder, f'{date}.json')
            # Open the input file and create the output JSON file
            with open(input_file, "r", encoding="utf-8") as infile, open(output_file, "w", encoding="utf-8") as outfile:
                json_list = []
                for line in infile:
                    try:
                        # Parse the line as JSON
                        parsed_line = json.loads(line.strip())
                        json_list.append(parsed_line)
                    except json.JSONDecodeError as e:
                        print(f"Error decoding line in {input_file}: {e}")
                
                # Write the proper JSON to the output file
                json.dump(json_list, outfile, indent=4)